# Analyzing a single speed skydive

In [1]:
from ssscoring import convertFlySight2SSScoring
from ssscoring import dropNonSkydiveDataFrom
from ssscoring import getSpeedSkydiveFrom
from ssscoring import isValidJump
from ssscoring import jumpAnalysisTable

import io

import bokeh.plotting as bp
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd

# Initializations:
bp.output_notebook(hide_banner = True)

## Get the data

In [3]:
uploader = widgets.FileUpload(description = 'Speed CSV', tooltip = 'FlySight speed file')
display(uploader)

FileUpload(value=(), description='Speed CSV', tooltip='FlySight speed file')

## Results

In [14]:
def processJump(data):
    data = data.copy()
    data = dropNonSkydiveDataFrom(data)
    window, data = getSpeedSkydiveFrom(data)
    validJump = isValidJump(data, window)
    score = 0.0
    
    if validJump:
        maxSpeed, table = jumpAnalysisTable(data)
        display(table)
        color = '#0f0'
        result = '🟢 valid'
        baseTime = data.iloc[0].timeUnix
        data['plotTime'] = data.timeUnix-baseTime
    
        scores = dict()  
        for spot in data.plotTime:
            r0 = data[data.plotTime == spot]
            r1 = data[data.plotTime == spot+3.0]
        
            if not r1.empty:
                scores[0.5*(float(r0.vKMh.iloc[0])+float(r1.vKMh.iloc[0]))] = spot
        score = max(scores)

    else:
        color = '#f00'
        result = '🔴 invalid'

    return (score, maxSpeed, scores, data, window, table, color, result)

In [15]:
data = convertFlySight2SSScoring(pd.read_csv(io.BytesIO(uploader.value[0].content), skiprows= (1,1)))
score, maxSpeed, scores, data, window, table, color, result = processJump(data)
validJumpStatus = '<hr><h1><span style="color: %s">%s jump - score = %.02f km/h</span></h1>' % (color, result, score)    

,time,vKMh,altitude (ft)
15281,5.0,175.176,13337.121283
15331,10.0,311.940,12200.809642
15381,15.0,396.000,10568.486971
15431,20.0,447.624,8628.576178
15481,25.0,358.668,6628.885927
15490,25.9,312.120,6357.895128


In [6]:
p = bp.figure(title = 'speed skydive',
              height = 300,
              width = 600,
              x_axis_label = 'seconds from exit',
              y_axis_label = 'km/h',
              x_range = (0, 30),
              y_range = (0, 550))

In [7]:
display(widgets.HTML(validJumpStatus))
if 'valid' in result:
    display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), breakoff at %d m (%d ft)</h3>'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
    display(table)
    p.line(data.plotTime, data.vKMh, legend_label = 'speed', line_width = 2, line_color = 'green')
    p.segment(x0 = [ scores[score], scores[score]+3.0, ], y0 = [ 0.0, 0.0, ], x1 = [ scores[score], scores[score]+3.0, ], y1 = [ maxSpeed, maxSpeed, ])
    bp.show(p)

HTML(value='<hr><h1><span style="color: #0f0">🟢 valid jump - score = 450.16 km/h</span></h1>')

HTML(value='<h3>Max speed = 456; exit at 4191 m (13752 ft), breakoff at 1935 m (6351 ft)</h3>')

,time,vKMh,altitude (ft)
15281,5.0,175.176,13337.121283
15331,10.0,311.940,12200.809642
15381,15.0,396.000,10568.486971
15431,20.0,447.624,8628.576178
15481,25.0,358.668,6628.885927
15490,25.9,312.120,6357.895128
